In [3]:
from flask import Flask, request, jsonify
import json
from time import time
from textwrap import dedent
from uuid import uuid4

In [4]:
import hashlib
import json
from time import time

In [5]:
# transactions > block > (pow) > chain => 모든 node가 동일한 chain을 공유

In [6]:
class BlockChain(object):
    
    def __init__(self):  # 생성자 함수 : 인스턴스(객체)가 만들어질 때 동시에 무조건 실행...
        self.chain = []   # 블록을 담을 체인 : 블록체인
        self.current_transaction = []   # 거래 내역을 담는 리스트
        self.nodes = set()  # 사용자 계정(주소)를 저장하는 set
        
        # genesis block(체인에서 최소 생성되는 블록)이 인스턴스 생성시 즉시 생성
        self.new_block(previouse_hash=1, proof=100)  
        
    def new_transaction(self, sender, recipient, amount): # 거래 내역을 담는 리스트에 거래 정보 저장
        
        self.current_transaction.append(
            {
                'sender':sender,  # 송신자
                'recipient':recipient,  # 수신자
                'amount':amount  # 금액
            }
        )
        
        return self.last_block['index'] + 1
    
    # 블록을 생성하고 체인에 블록을 추가하는 함수..
    # proof : 채굴에 성공했을 때, nounce값 
    # 채굴에 성공해야만 블록을 생성하고 chain에 추가가 가능
    def new_block(self, previous_hash=None, proof): # 거래내역, 이전블록헤쉬, proof
        
        block = {  # 블록 생성
            'index':len(self.chain) + 1,   # 블록의 인덱스값
            'timestamp': str(datetime.datetime.now()),  # 블록이 생성되는 시간
            'transactions' : self.current_transaction,   # 거래 내역이 저장된 리스트
            'proof': proof,  # 채굴에 성공해서 얻은 nounce값
            'previous_hash': previous_hash or self.hash(self.chain[-1])  # 이전 블록의 해쉬값
        }
        
        self.current_transaction = []  # 거래내역 리스트를 비운다
        
        self.chain.append(block)  # 블록을 체인에 추가
        
        return block
    
    def register_node(self, address):  # 계정(주소)들을 nodes에 추가
        parsed_url = urlparse(address)
        self.nodes.add(parsed_url.netloc)
        
    def valid_chain(self, chain):  # 블록체인이 유효한지 검증
        last_block = chain[0]  # 최초의 블록
        current_index =1  # 그 다음 블록인덱스
        
        while current_index < len(chain):  # 전체 체인을 반복해서 검증
            current_block = chain[current_index]  # 현재 블록
            
            # 현재 블록에 등록된 이전블록 해쉬값 == 이전블록을 해쉬함수를 통해 직접 계산한 해쉬값
            if current_block['previous_hash'] != self.hash(last_block):
                return False  # 동일하지 않으면 블록의 유효성은 False
            
            last_block = block  # 현재 블록을 이전 블록으로
            current_index += 1  # 다음 블록 인덱스
            
        return True  # 블록이 유효하다
    
    # 어느 블록이 길이가 더 길면서 유효한 블록이면 현재의 블록을 더 길고 유효한 블록을 교체
    def resolve_conflicts(self):  
        neighbors = self.nodes  # 사용자 계정(주소)정보가 인는 set
        new_chain = None
        
        my_chain_length = len(self.my_chain)  # 현재 나의 계정의 블록체인 길이
        
        for node in neighbors:
            tmp_url = 'http://' + str(node) + '/chain'   # 다른 사용자들의 계정 주소의 chain경로
            response = requests.get(tem_url)  # get방식으로 요청 => flask가 chain데이터, 체인 길이 리턴
                        
            if response.status_code == 200:   # 정상 통신
                user_length = response.json()['length']  # json => diot변환, 길이정보를 받은
                user_chain = response.json()['chain']  # json => diot변환, chain을 받음
                
                if user_length > my_chain_length and self.valid_chain(user_chain):
                    my_chain_length = user_length
                    new_chain = user_chain
                    
            if new_chain:
                self.my_chain = new_chain
                return True
            
            return False  # 내 체인이 교체되지 않았ㄷㅏ. 상대방 체인이 더 길지 않았음...
        
    def hash(block):
        # dump : dict => json(str) 
        # encoding : json(str) => binary
        block_string = json.dumps(block, sort_keys=True).encode()  # dump : dict => json(str) => binary
        return hashlib.sha256(block_string).hexdigest()  # hash값 출략
    
    def last_block(self):  # chain에 block이 append(최근 블록이 맨 뒤에 뒤치)
        return self.chain[-1]  # 가장 최근의 block를 리턴
    
    def pow(self, last_proof):  # 채굴 : 조건에 맞는 해쉬를 구할 때 까지 반복해서 해쉬값을 구하는 함수
        
        proof = 0
        
        while self.valid_proof(last_proof, proof) is False:
            proof += 1
            
        return proof  # 문제를 풀었을 때(채굴에 성공한 경우)의 proof값(nounce값)
    
    def valid_proof(last_proof, proof):  # 블록이 해쉬값이 문제(조건)를 풀었는지 확인
        # 원래 : 블록의 헤더와 새로운 proof를 통해서 조건에 맞는 해쉬값을 찾아야..
        # 현재코드 : 이전의 proof값과 새로운 proof를 더한 값으로 조건에 맞는 해쉬값을 찾는다..(단순화)
        
        guess = str(last_proof + proof).encode()
        guess_hash = hashlib.sha256(guess).hexdigest()
        return guess_hash[:4] == '0000'  # 채굴의 문제(조건) : 해쉬값의 앞 4자리가 0인지..
        # 채굴의 난이도 : 몇 자리까지를 0으로 해야 하는가에 따라 달라짐

SyntaxError: non-default argument follows default argument (Temp/ipykernel_7020/1180483081.py, line 22)

In [1]:
app = Flask(__name__)

node_identifier = str(uuid4()).replace('-','')  # uuid4() : unique한 값을 생성, 최고의 계정 주소

blockchain = BlockChain() 
# 블록체인 클래스 객체(인스턴스) 생성 -> 생성자함수 실행 : chain, transaction list, nodes, genesis block

@app.route('/mine', methods=['GET'])  # get 방식으로 /mine 경로로 접근
def mine():
    last_block = blockchain.last_block  # 가장 최근 블록
    last_proof = last_block['proof']  # 이전 블록에서 채굴 성공으로 얻은 proof값
    
    proof = blockchain.pow(last_proof)  # 현재 블록에서 채굴 성공으로 얻은 새로은 proof값
    
    blockchain.new_transaction(  # 거래 정보를 저장하는 함수
        sender = '0',   # 채굴을 통한 것이므로 전달자가 없다
        recipient = node_identifier,  # 랜덤한 수신자
        amount = 1 # 1코인
    )
    
    previous_hash = blockchain.hash(last_block)  # 이전 블록을 해쉬한 값
    
    block = blockchain.new_block(proof, previous_hash)  # 새로운 블록을 생성하고 체인에 추가
    
    response = {
        'message': 'new_block_found',
        'index': block['index'],
        'transactions': block['transactions'],
        'proof': block['proof'],
        'previous_hash' : block['previouse_hash']
    }
    
    return jsonfy(response), 200


@app.route('/chain', methods=['GET'])  # /chain으로 get방식으로 요청이 들어오면
def full_chain():
    
    response = {
        'chain': blockchain.chain,
        'length': len(blockchain.chain),
    }
    
    return jsonify(response), 200  # 체인데이터와 체인길이 정보, 통신 응답 코드를 리턴

@app.route('/nodes/register', methods=['POST'])  # post방식으로 요청이 들어오면
def register_nodes():
    values = request.get_json()  # json으로 들어온 데이터를 수신
    nodes = values.get('nodes')  # json에서 nodes정보를 저장, nodes에는 계정정보가 있음
    
    if nodes is None:
        return 'Error : please supply a valid list of nodes', 400
    
    for node in nodes:
        blockchain.register_node(node)  # 계정(주소)정보를 nodes라는 계정 변수에 저장
        
    response = {
        'message' : 'New nodes have been added',
        'total_nodes' : list(blockchain.nodes)
    }
    
    return jsonify(response), 201  # dict정보를 json형식으로 리턴

@app.route('/nodes/resolve', methods= ['GET'])  # /nodes/resolve 경로로 get방식으로 접근(요청)이 오면
def consensus():
    replaced = blockchain.resolve_conflicts()  # 내 chain이 없데이트 되면 True, 아니면 False
    
    if replaced:
        response = {
            'message':'Our chain was replaced',
            'new chain' : blockchain.chain
        }
    else:
        response = {
            'message':'Our chain is valid',
            'new chain' : blockchain.chain
        }    
        
    return jsonify(response), 200

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=8080)
        
    



NameError: name 'Flask' is not defined